In [1]:
import sys
sys.path.append(r"D:\Certification\Bloc_6\src")   

from data_loader import get_clean_data
from load_to_db import load_dim_tables, load_fact_tables
from db import engine


d:\Certification\Bloc_6\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import sys
sys.path.append(r"D:\Certification\Bloc_6\src")   

from data_loader import get_clean_data
from load_to_db import load_dim_tables, load_fact_tables
from db import engine


In [3]:
# Télécharger et nettoyer les données puis enregistrer CSV
df_clean = get_clean_data(save_csv=False)  # False  pour ne pas enregistrer CSV une deuxième fois 


Encodage détecté: {'encoding': 'Windows-1252', 'confidence': 0.73, 'language': ''}


In [4]:
import pandas as pd

#   Création de table dim_date   
df_dim_date = pd.DataFrame({
    "date_id": pd.to_datetime(df_clean["order_date"].unique())
})
df_dim_date["year"] = df_dim_date["date_id"].dt.year
df_dim_date["quarter"] = df_dim_date["date_id"].dt.quarter
df_dim_date["month"] = df_dim_date["date_id"].dt.month
df_dim_date["month_name"] = df_dim_date["date_id"].dt.month_name()
df_dim_date["week"] = df_dim_date["date_id"].dt.isocalendar().week
df_dim_date["day"] = df_dim_date["date_id"].dt.day
df_dim_date["day_name"] = df_dim_date["date_id"].dt.day_name()
df_dim_date["is_weekend"] = df_dim_date["date_id"].dt.weekday >= 5


In [5]:
load_dim_tables(df_clean, df_dim_date)


In [6]:
load_fact_tables(df_clean)


✅ Tables chargées dans la DB


In [7]:
from sqlalchemy import inspect

#  Création d'un (inspector) pour engine
inspector = inspect(engine)

#  List des tables  
tables = inspector.get_table_names()
print("✅ Tables existant dans la BD   :")
for t in tables:
    print("-", t)


✅ Tables existant dans la BD   :
- customers
- locations
- products
- dim_date
- sales
- cleaned_data
- orders


In [8]:
for table_name in tables:
    print(f"\n📌 les columns de la table  {table_name}:")
    columns = inspector.get_columns(table_name)
    for col in columns:
        print(f" - {col['name']} ({col['type']})")



📌 les columns de la table  customers:
 - customer_id (TEXT)
 - customer_name (TEXT)
 - segment (TEXT)

📌 les columns de la table  locations:
 - location_id (BIGINT)
 - country (TEXT)
 - city (TEXT)
 - state (TEXT)
 - postal_code (TEXT)
 - region (TEXT)

📌 les columns de la table  products:
 - product_id (TEXT)
 - product_name (TEXT)
 - category (TEXT)
 - subcategory (TEXT)

📌 les columns de la table  dim_date:
 - date_id (TIMESTAMP)
 - year (INTEGER)
 - quarter (INTEGER)
 - month (INTEGER)
 - month_name (TEXT)
 - week (BIGINT)
 - day (INTEGER)
 - day_name (TEXT)
 - is_weekend (BOOLEAN)

📌 les columns de la table  sales:
 - order_id (TEXT)
 - customer_id (TEXT)
 - product_id (TEXT)
 - location_id (BIGINT)
 - sales (DOUBLE PRECISION)
 - quantity (BIGINT)
 - discount (DOUBLE PRECISION)
 - profit (DOUBLE PRECISION)
 - order_date (TIMESTAMP)
 - ship_date (TIMESTAMP)
 - ship_mode (TEXT)

📌 les columns de la table  cleaned_data:
 - order_id (TEXT)
 - order_date (TIMESTAMP)
 - ship_date (TIME

In [9]:
import pandas as pd

# Afficher les 5 premieres lignes de la table sales
df_sales = pd.read_sql("SELECT * FROM sales LIMIT 5;", engine)
df_sales


,order_id,customer_id,product_id,location_id,sales,quantity,discount,profit,order_date,ship_date,ship_mode
0,CA-2016-152156,CG-12520,FUR-BO-10001798,66763,261.9600,2,0.00,41.9136,2016-11-08,2016-11-11,Second Class
1,CA-2016-152156,CG-12520,FUR-CH-10000454,66763,731.9400,3,0.00,219.5820,2016-11-08,2016-11-11,Second Class
2,CA-2016-138688,DV-13045,OFF-LA-10000240,20305,14.6200,2,0.00,6.8714,2016-06-12,2016-06-16,Second Class
3,US-2015-108966,SO-20335,FUR-TA-10000577,81064,957.5775,5,0.45,-383.0310,2015-10-11,2015-10-18,Standard Class
4,US-2015-108966,SO-20335,OFF-ST-10000760,81064,22.3680,2,0.20,2.5164,2015-10-11,2015-10-18,Standard Class


In [10]:
df_customers = pd.read_sql("SELECT * FROM customers LIMIT 5;", engine)
df_customers

,customer_id,customer_name,segment
0,CG-12520,Claire Gute,Consumer
1,DV-13045,Darrin Van Huff,Corporate
2,SO-20335,Sean O'Donnell,Consumer
3,BH-11710,Brosina Hoffman,Consumer
4,AA-10480,Andrew Allen,Consumer


In [11]:
df_locations = pd.read_sql("SELECT * FROM locations LIMIT 5;", engine)
df_locations

,location_id,country,city,state,postal_code,region
0,66763,United States,Henderson,Kentucky,42420,South
1,20305,United States,Los Angeles,California,90036,West
2,81064,United States,Fort Lauderdale,Florida,33311,South
3,32580,United States,Los Angeles,California,90032,West
4,65928,United States,Concord,North Carolina,28027,South


In [12]:
df_products = pd.read_sql("SELECT * FROM products LIMIT 5;", engine)
df_products

,product_id,product_name,category,subcategory
0,FUR-BO-10001798,Bush Somerset Collection Bookcase,Furniture,Bookcases
1,FUR-CH-10000454,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",Furniture,Chairs
2,OFF-LA-10000240,Self-Adhesive Address Labels for Typewriters b...,Office Supplies,Labels
3,FUR-TA-10000577,Bretford CR4500 Series Slim Rectangular Table,Furniture,Tables
4,OFF-ST-10000760,Eldon Fold 'N Roll Cart System,Office Supplies,Storage


In [13]:
df_dim_date = pd.read_sql("SELECT * FROM dim_date LIMIT 5;", engine)
df_dim_date

,date_id,year,quarter,month,month_name,week,day,day_name,is_weekend
0,2016-11-08,2016,4,11,November,45,8,Tuesday,False
1,2016-06-12,2016,2,6,June,23,12,Sunday,True
2,2015-10-11,2015,4,10,October,41,11,Sunday,True
3,2014-06-09,2014,2,6,June,24,9,Monday,False
4,2017-04-15,2017,2,4,April,15,15,Saturday,True


In [14]:
df_orders = pd.read_sql("SELECT * FROM orders LIMIT 5;", engine)
df_orders

,order_id,order_date,ship_date,ship_mode,shipping_time_days,date_id
0,CA-2016-152156,2016-11-08,2016-11-11,Second Class,3,2016-11-08
1,CA-2016-138688,2016-06-12,2016-06-16,Second Class,4,2016-06-12
2,US-2015-108966,2015-10-11,2015-10-18,Standard Class,7,2015-10-11
3,CA-2014-115812,2014-06-09,2014-06-14,Standard Class,5,2014-06-09
4,CA-2017-114412,2017-04-15,2017-04-20,Standard Class,5,2017-04-15
